<a href="https://colab.research.google.com/github/Tyuhdj/Recommendation-System/blob/main/Recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**GATED RECURRENT UNIT NETWORK MODEL(RNN)**

**INSTALL AND IMPORT LIBRARIES**

In [1]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 1.3 MB/s eta 0:00:00


In [2]:
from typing import Dict, Text

import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs
import matplotlib.pyplot as plt

**READ THE DATA**

In [3]:
# Ratings data.
ratings = tfds.load('movielens/100k-ratings', split="train")
# Features of all the available movies.
movies = tfds.load('movielens/100k-movies', split="train")

# Select the basic features.
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"]
})
movies = movies.map(lambda x: x["movie_title"])

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/100000 [00:00<?, ? examples/s]

...

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/1682 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/movielens/100k-movies/incomplete.PCKJ1S_0.1.1/movielens-train.tfrecord*...…

Dataset movielens downloaded and prepared to /root/tensorflow_datasets/movielens/100k-movies/0.1.1. Subsequent calls will reuse this data.


In [4]:
print("Ratings Dataset:")
for rating in ratings.take(5):
    print(f"User ID: {rating['user_id'].numpy()}")
    print(f"Movie Title: {rating['movie_title'].numpy()}")
    print("---")


Ratings Dataset:
User ID: b'138'
Movie Title: b"One Flew Over the Cuckoo's Nest (1975)"
---
User ID: b'92'
Movie Title: b'Strictly Ballroom (1992)'
---
User ID: b'301'
Movie Title: b'Very Brady Sequel, A (1996)'
---
User ID: b'60'
Movie Title: b'Pulp Fiction (1994)'
---
User ID: b'197'
Movie Title: b'Scream 2 (1997)'
---


**BUILD VOCABULARIES**

In [5]:
user_ids_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
user_ids_vocabulary.adapt(ratings.map(lambda x: x["user_id"]))

movie_titles_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
movie_titles_vocabulary.adapt(movies)

 **DEFINE MODEL**

In [6]:
class MovieLensModel(tfrs.Model):
  # We derive from a custom base class to help reduce boilerplate. Under the hood,
  # these are still plain Keras Models.

  def __init__(
      self,
      user_model: tf.keras.Model,
      movie_model: tf.keras.Model,
      task: tfrs.tasks.Retrieval):
    super().__init__()

    # Set up user and movie representations.
    self.user_model = user_model
    self.movie_model = movie_model

    # Set up a retrieval task.
    self.task = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # Define how the loss is computed.

    user_embeddings = self.user_model(features["user_id"])
    movie_embeddings = self.movie_model(features["movie_title"])

    return self.task(user_embeddings, movie_embeddings)
ratings = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)
train_size = int(0.8 * 100_000)
train = ratings.take(train_size)
test = ratings.skip(train_size).take(100_000 - train_size)


**DEFINING MODEL AND THE RETRIEVAL TASK**

In [7]:
user_model = tf.keras.Sequential([
    user_ids_vocabulary,
    tf.keras.layers.Embedding(user_ids_vocabulary.vocab_size(), 128),  # Embedding dimension
    tf.keras.layers.Reshape((1, 128)),  # Reshape to add the sequence dimension
    tf.keras.layers.GRU(64, return_sequences=True),
    tf.keras.layers.GRU(32)
])

movie_model = tf.keras.Sequential([
    movie_titles_vocabulary,
    tf.keras.layers.Embedding(movie_titles_vocabulary.vocab_size(), 128),  # Embedding dimension
    tf.keras.layers.Reshape((1, 128)),  # Reshape to add the sequence dimension
    tf.keras.layers.GRU(64, return_sequences=True),
    tf.keras.layers.GRU(32)
])

# Define your objectives.
task = tfrs.tasks.Retrieval(metrics=tfrs.metrics.FactorizedTopK(
    movies.batch(128).map(movie_model)
))

**FIT AND EVALUATE**

In [8]:
# Create a retrieval model.
model = MovieLensModel(user_model, movie_model, task)
model.compile(optimizer=tf.keras.optimizers.Adam(0.001))  # Adam optimizer

# Train for more epochs and store training history.
history1 = model.fit(
    train.batch(4096),
    validation_data=test.batch(4096),
    epochs=10
)

# Use brute-force search to set up retrieval using the trained representations.
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index_from_dataset(
    movies.batch(100).map(lambda title: (title, model.movie_model(title))))


Epoch 1/10
20/20 [==============================] - 40s 1s/step - factorized_top_k/top_1_categorical_accuracy: 2.1250e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0023 - factorized_top_k/top_10_categorical_accuracy: 0.0054 - factorized_top_k/top_50_categorical_accuracy: 0.0288 - factorized_top_k/top_100_categorical_accuracy: 0.0575 - loss: 32417.5411 - regularization_loss: 0.0000e+00 - total_loss: 32417.5411 - val_factorized_top_k/top_1_categorical_accuracy: 3.0000e-04 - val_factorized_top_k/top_5_categorical_accuracy: 0.0022 - val_factorized_top_k/top_10_categorical_accuracy: 0.0055 - val_factorized_top_k/top_50_categorical_accuracy: 0.0286 - val_factorized_top_k/top_100_categorical_accuracy: 0.0568 - val_loss: 29626.3398 - val_regularization_loss: 0.0000e+00 - val_total_loss: 29626.3398
Epoch 2/10
20/20 [==============================] - 27s 1s/step - factorized_top_k/top_1_categorical_accuracy: 6.2500e-05 - factorized_top_k/top_5_categorical_accuracy: 0.0054 - factorized_top

In [9]:
# Get some recommendations for user with ID "42".
_, titles = index(np.array(["42"]))
print(f"Top 3 recommendations for user 42: {titles[0, :10]}")

Top 3 recommendations for user 42: [b'Cliffhanger (1993)' b'Getaway, The (1994)'
 b'Conan the Barbarian (1981)' b'Indian in the Cupboard, The (1995)'
 b'Secret Adventures of Tom Thumb, The (1993)' b'Sliver (1993)'
 b'Star Trek VI: The Undiscovered Country (1991)' b'Under Siege (1992)'
 b'Speed (1994)' b'Blue Chips (1994)']


**NEURAL COLLABORATIVE FILTERING(NCF) MODEL**

**INSTALL AND IMPORT LIBRARIES**

In [18]:
!pip install -q tensorflow-recommenders
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs
import numpy as np

**READ THE DATASET**

In [19]:
# Load the MovieLens dataset
ratings = tfds.load("movielens/100k-ratings", split="train")
movies = tfds.load("movielens/100k-movies", split="train")

In [20]:
# Prepare the data
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"]
})

movies = movies.map(lambda x: x["movie_title"])


**DEFINE MODEL AND RETRIEVAL TASK**

In [21]:
# Define the user and movie model.
user_ids_vocabulary = tf.keras.layers.StringLookup()
movie_titles_vocabulary = tf.keras.layers.StringLookup()

user_ids_vocabulary.adapt(ratings.map(lambda x: x["user_id"]))
movie_titles_vocabulary.adapt(movies)

user_model = tf.keras.Sequential([
    user_ids_vocabulary,
    tf.keras.layers.Embedding(user_ids_vocabulary.vocab_size(), 64)
])

movie_model = tf.keras.Sequential([
    movie_titles_vocabulary,
    tf.keras.layers.Embedding(movie_titles_vocabulary.vocab_size(), 64)
])

In [22]:
# Define the NCF model.
class NCFModel(tfrs.Model):
    def __init__(self, user_model, movie_model, task):
        super().__init__()
        self.user_model = user_model
        self.movie_model = movie_model
        self.task = task

        # Neural network layers
        self.dense_layers = tf.keras.Sequential([
            tf.keras.layers.Dense(128, activation="relu"),
            tf.keras.layers.Dense(64, activation="relu"),
            tf.keras.layers.Dense(32, activation="relu"),
            tf.keras.layers.Dense(1)
        ])

    def compute_loss(self, features, training=False):
        user_embeddings = self.user_model(features["user_id"])
        movie_embeddings = self.movie_model(features["movie_title"])

        # Concatenate user and movie embeddings
        concatenated_embeddings = tf.concat([user_embeddings, movie_embeddings], axis=1)

        # Pass through dense layers
        output = self.dense_layers(concatenated_embeddings)

        return self.task(user_embeddings, movie_embeddings)


In [23]:
# Define the retrieval task.
task = tfrs.tasks.Retrieval(metrics=tfrs.metrics.FactorizedTopK(
    movies.batch(128).map(movie_model)
))


**FIT AND EVALUATE**

In [24]:
# Create and compile the model.
model = NCFModel(user_model, movie_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))


In [25]:
# Train the model.
model.fit(ratings.batch(4096), epochs=5)

Epoch 1/5


25/25 [==============================] - 18s 674ms/step - factorized_top_k/top_1_categorical_accuracy: 1.0000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0016 - factorized_top_k/top_10_categorical_accuracy: 0.0047 - factorized_top_k/top_50_categorical_accuracy: 0.0431 - factorized_top_k/top_100_categorical_accuracy: 0.0970 - loss: 33085.0657 - regularization_loss: 0.0000e+00 - total_loss: 33085.0657
Epoch 2/5
25/25 [==============================] - 15s 581ms/step - factorized_top_k/top_1_categorical_accuracy: 1.1000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0047 - factorized_top_k/top_10_categorical_accuracy: 0.0142 - factorized_top_k/top_50_categorical_accuracy: 0.1051 - factorized_top_k/top_100_categorical_accuracy: 0.2099 - loss: 31012.6473 - regularization_loss: 0.0000e+00 - total_loss: 31012.6473
Epoch 3/5
25/25 [==============================] - 15s 591ms/step - factorized_top_k/top_1_categorical_accuracy: 2.7000e-04 - factorized_top_k/top_5_categorical_accur

In [26]:
# Set up brute-force search for retrieval.
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index_from_dataset(
    movies.batch(100).map(lambda title: (title, model.movie_model(title)))
)

# Get recommendations.
_, titles = index(np.array(["100"]))
print(f"Top 3 recommendations for user 100: {titles[0, :3]}")


Top 3 recommendations for user 100: [b'Big Bang Theory, The (1994)' b'N\xc3\xa9nette et Boni (1996)'
 b'Other Voices, Other Rooms (1997)']
